<a href="https://colab.research.google.com/github/Data-Science-and-Data-Analytics-Courses/MITx---Machine-Learning-with-Python-From-Linear-Models-to-Deep-Learning-Jun-11-2019/blob/master/Unit%202%20Nonlinear%20Classification%2C%20Linear%20regression%2C%20Collaborative%20Filtering%20(2%20weeks)/Homeworks/Homework_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 3

---



## Setup

In [1]:
import os, sys
from pathlib import Path
from google.colab import drive

# Notebook Library
url = "https://github.com/Data-Science-and-Data-Analytics-Courses/Notebook-Library"
repo = Path("/nblib")
!git clone "{url}" "{repo}"
if repo.parent.as_posix() not in sys.path:
  sys.path.append(repo.parent.as_posix())
%run "{repo}/.Importable.ipynb"

from nblib import Git
# Remote
URL = "https://github.com/Data-Science-and-Data-Analytics-Courses/MITx---Machine-Learning-with-Python-From-Linear-Models-to-Deep-Learning-Jun-11-2019"
REPO = Git.clone(URL, dest="/content")
if REPO.as_posix() not in sys.path:
  sys.path.append(REPO.as_posix())

# Working directory
NBDIR = REPO / "Unit 2 Nonlinear Classification, Linear regression, Collaborative Filtering (2 weeks)/Homeworks/"
os.chdir(NBDIR)

from setup.Setup import *

Cloning into '/nblib'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 32 (delta 15), reused 7 (delta 2), pack-reused 0
Unpacking objects: 100% (32/32), done.


importing Jupyter notebook from /nblib/Git.ipynb
Cloning into '/content/MITx---Machine-Learning-with-Python-From-Linear-Models-to-Deep-Learning-Jun-11-2019'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 434 (delta 44), reused 13 (delta 5), pack-reused 340
Receiving objects: 100% (434/434), 5.50 MiB | 8.16 MiB/s, done.
Resolving deltas: 100% (236/236), done.
importing Jupyter notebook from /content/MITx---Machine-Learning-with-Python-From-Linear-Models-to-Deep-Learning-Jun-11-2019/setup/Setup.ipynb


## Collaborative Filtering, Kernels, Linear Regression
In this question, we will use the alternating projections algorithm for low-rank matrix factorization, which aims to minimize

$$
J(U, V) = \underbrace{\frac{1}{2}\sum _{(a,i) \in D} (Y_{ai} - [UV^ T]_{ai})^2}_{\text {Squared Error}} + \underbrace{\frac{\lambda }{2}\sum _{a=1}^ n \sum _{j=1}^ k U_{aj}^2 + \frac{\lambda }{2}\sum _{i=1}^ m \sum _{j=1}^ k V_{ij}^2}_{\text {Regularization}}.
$$

In the following, we will call the first term the squared error term, and the two terms with  λ  the regularization terms.

Let  Y  be defined as

$$
Y = \begin{bmatrix}  5 &  ? &  7 \\ ? &  2 &  ? \\ 4 &  ? &  ? \\ ? &  3 &  6 \\ \end{bmatrix}
$$

D  is defined as the set of indices (a,i), where Ya,i is not missing. In this problem, we let k=λ=1. Additionally, U and V are initialized as U(0)=[6,0,3,6]T, and V(0)=[4,2,1]T.  


Compute  X , the matrix of predicted rankings  UVT  given the initial values for  U  and  V .  


In [2]:
u = np.array([6,0,3,6])
v = np.array([4,2,1])
x = np.outer(u, v)
print(x)

[[24 12  6]
 [ 0  0  0]
 [12  6  3]
 [24 12  6]]


Compute the squared error term, and the regularization terms in for the current estimate  X .  


In [194]:
la = 1 # lambda
y = ma.masked_invalid([[5, np.nan, 7],
                      [np.nan, 2, np.nan],
                      [4, np.nan, np.nan],
                      [np.nan, 3, 6]])
err_sq = 1/2 * np.sum((y-x)**2)
reg = 1/2 * la * (np.sum(u**2) + np.sum(v**2))
print(err_sq)
print(reg)

255.5
51.0


Suppose  V  is kept fixed. Run one step of the algorithm to find the new estimate  U(1) .

Enter the  U(1)  as a list of numbers,  [U(1)1,U(1)2,U(1)3,U(1)4] :  


In [182]:
U = sym.Matrix(sym.symarray('U', (1, 4)))
V = sym.Matrix(sym.symarray('V', (1, 3)))
Y = sym.Matrix(sym.symarray('Y', (4, 3)))
Known = sym.Matrix([[1, np.nan, 1],
                    [np.nan, 1, np.nan],
                    [1, np.nan, np.nan],
                    [np.nan, 1, 1]])
Y = matrix_multiply_elementwise(Y,Known)

X = U.T * V
Z = Y - X
Z = sym.simplify(1/2 * matrix_multiply_elementwise(Z,Z)) # squared error component matrix
sym.pprint(Z)

RegU = 1/2 * sym.Symbol("lamda") * matrix_multiply_elementwise(U, U) # regularization of U
sym.pprint(RegU)

RegV = 1/2 * sym.Symbol("lamda") * matrix_multiply_elementwise(V, V) # regularization of V
sym.pprint(RegV)

⎡                      2                                                 2⎤
⎢0.5⋅(U₀ ₀⋅V₀ ₀ - Y₀ ₀)             nan            0.5⋅(U₀ ₀⋅V₀ ₂ - Y₀ ₂) ⎥
⎢                                                                         ⎥
⎢                                               2                         ⎥
⎢          nan            0.5⋅(U₀ ₁⋅V₀ ₁ - Y₁ ₁)             nan          ⎥
⎢                                                                         ⎥
⎢                      2                                                  ⎥
⎢0.5⋅(U₀ ₂⋅V₀ ₀ - Y₂ ₀)             nan                      nan          ⎥
⎢                                                                         ⎥
⎢                                               2                        2⎥
⎣          nan            0.5⋅(U₀ ₃⋅V₀ ₁ - Y₃ ₁)   0.5⋅(U₀ ₃⋅V₀ ₂ - Y₃ ₂) ⎦
⎡        2            2            2            2  ⎤
⎣0.5⋅U₀ ₀ ⋅λ  0.5⋅U₀ ₁ ⋅λ  0.5⋅U₀ ₂ ⋅λ  0.5⋅U₀ ₃ ⋅λ⎦
⎡        2            2            2  ⎤
⎣0.5⋅V₀ ₀ ⋅λ  0.5⋅

In [210]:
subs = {"V_0_0": v[0], "V_0_1": v[1], "V_0_2": v[2],
        "Y_0_0": y[0, 0], "Y_0_2": y[0, 2],
        "Y_1_1": y[1, 1],
        "Y_2_0": y[2, 0],
        "Y_3_1": y[3, 1], "Y_3_2": y[3, 2],
        "lamda": la}
# U(1)
i = sym.Symbol("i")
j = sym.Symbol("j")
for k in range(4):
  difZ = Z.diff(U[0, k])
  err_sq = sym.Sum(difZ[i, j], (i, 0, 3), (j, 0, 2)).doit()
  sym.pprint(err_sq)

  difRegU = RegU.diff(U[0, k])
  reg = sym.Sum(difRegU[i, j], (i, 0, 0), (j, 0, 3)).doit()
  sym.pprint(reg)

  sym.pprint(sym.solve(err_sq+reg, U[0,k]))
  print(sym.solve(err_sq+reg, U[0,k])[0].subs(subs).evalf())

1.0⋅V₀ ₀⋅(U₀ ₀⋅V₀ ₀ - Y₀ ₀) + 1.0⋅V₀ ₂⋅(U₀ ₀⋅V₀ ₂ - Y₀ ₂)
1.0⋅U₀ ₀⋅λ
⎡V₀ ₀⋅Y₀ ₀ + V₀ ₂⋅Y₀ ₂⎤
⎢─────────────────────⎥
⎢      2       2      ⎥
⎣  V₀ ₀  + V₀ ₂  + λ  ⎦
1.50000000000000
1.0⋅V₀ ₁⋅(U₀ ₁⋅V₀ ₁ - Y₁ ₁)
1.0⋅U₀ ₁⋅λ
⎡V₀ ₁⋅Y₁ ₁⎤
⎢─────────⎥
⎢    2    ⎥
⎣V₀ ₁  + λ⎦
0.800000000000000
1.0⋅V₀ ₀⋅(U₀ ₂⋅V₀ ₀ - Y₂ ₀)
1.0⋅U₀ ₂⋅λ
⎡V₀ ₀⋅Y₂ ₀⎤
⎢─────────⎥
⎢    2    ⎥
⎣V₀ ₀  + λ⎦
0.941176470588235
1.0⋅V₀ ₁⋅(U₀ ₃⋅V₀ ₁ - Y₃ ₁) + 1.0⋅V₀ ₂⋅(U₀ ₃⋅V₀ ₂ - Y₃ ₂)
1.0⋅U₀ ₃⋅λ
⎡V₀ ₁⋅Y₃ ₁ + V₀ ₂⋅Y₃ ₂⎤
⎢─────────────────────⎥
⎢      2       2      ⎥
⎣  V₀ ₁  + V₀ ₂  + λ  ⎦
2.00000000000000
